## What is stacking?
Stacking or Stacked Generalization is an ensemble machine learning algorithm.

It uses a meta-learning algorithm to learn how to best combine the predictions from two or more base machine learning models.
Stacking is an ensemble learning technique that combines multiple classifications or regression models via a meta-classifier or a meta-regressor. 

The base-level models are trained based on a complete training set, then the meta-model is trained on the outputs of the base level model as features.
The base-level often consists of different learning algorithms and therefore stacking ensembles are often heterogeneous.

This notebook explains how we can go about explore and prepare data for model building.The notebook is structured in the following way

- About Dataset
- Data Summary
- Feature Engineering
- Missing Value Analysis
- Outlier Analysis
- Correlation Analysis
- Visualizing Distribution Of Data
- Visualizing Count Vs (Month,Season,Hour,Weekday,Usertype)
- Filling 0's In Windspeed Using Random Forest
- Linear Regression Model
- Regularization Models
- Ensemble Models

### About Dataset
### Overview
Bike sharing systems are a means of renting bicycles where the process of obtaining membership, rental, and bike return is automated via a network of kiosk locations throughout a city. Using these systems, people are able rent a bike from a one location and return it to a different place on an as-needed basis. Currently, there are over 500 bike-sharing programs around the world.

### Data Fields
* datetime - hourly date + timestamp
* season - 1 = spring, 2 = summer, 3 = fall, 4 = winter
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
weather -
    1. Clear, Few clouds, Partly cloudy, Partly cloudy
    1. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    1. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    1. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* count - number of total rentals (Dependent Variable)

## Load Library

In [ ]:
import pylab
import calendar

import numpy as np
import pandas as pd

import seaborn as sn
import matplotlib.pyplot as plt

from scipy import stats
import missingno as msno
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import scorer, mean_absolute_error, mean_squared_error, r2_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

pd.options.mode.chained_assignment = None

%matplotlib inline

## Load Dataset

In [ ]:
bike = pd.read_csv("../input/bike-sharing-demand/train.csv")

In [ ]:
# Show first 5 rows

bike.head(5)

In [ ]:
# size of the data
bike.shape

In [ ]:
# some basic information about data
bike.info()

## Feature Engineering

As we see from the above results, the columns "season","holiday","workingday" and "weather" should be of "categorical" data type.But the current data type is "int" for those columns. Let us transform the dataset in the following ways so that we can get started up with our EDA

* Create new columns "date,"hour","weekDay","month" from "datetime" column.
* Coerce the datatype of "season","holiday","workingday" and weather to category.
* Drop the datetime column as we already extracted useful features from it.

### Creating New Columns From "Datetime" Column


In [ ]:
bike["date"] = bike.datetime.apply(lambda x : x.split()[0])

bike["hour"] = bike.datetime.apply(lambda x : x.split()[1].split(":")[0])

bike["weekday"] = bike.date.apply(lambda dateString : calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])

bike["month"] = bike.date.apply(lambda dateString : calendar.month_name[datetime.strptime(dateString,"%Y-%m-%d").month])

bike["season"] = bike.season.map({1: "Spring", 2 : "Summer", 3 : "Fall", 4 :"Winter" })

bike["weather"] = bike.weather.map({1: " Clear + Few clouds + Partly cloudy + Partly cloudy",\
                                        2 : " Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist ", \
                                        3 : " Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds", \
                                        4 :" Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog " })

### Covert To Category Type

In [ ]:
categoryVariableList = ["hour","weekday","month","season","weather","holiday","workingday"]

for var in categoryVariableList:
    bike[var] = bike[var].astype("category")

### Dropping Unncessary Columns

In [ ]:
dailyData  = bike.drop(["datetime"],axis=1)

### Lets Start With Very Simple Visualization Of Variables DataType Count

In [ ]:
# count the data type in each column
bike_df = pd.DataFrame(bike.dtypes.value_counts()).reset_index().rename(columns={"index":"variableType",0:"count"})

bike_df

## Missing Values Analysis

Once we get hang of the data and columns, next step we geneally is to find out whether we have any missing values in our data. Luckily we dont have any missing value in the dataset. One way which I generally prefer to visualize missing value in the dataset is through "missingno".

In [ ]:
msno.matrix(bike,figsize=(12,5))

## Outliers Analysis

At first look, "count" variable contains lot of outlier data points which skews the distribution towards right (as there are more data points beyond Outer Quartile Limit).But in addition to that, following inferences can also been made from the simple boxplots given below.

* Spring season has got relatively lower count.The dip in median value in boxplot gives evidence for it.
* The boxplot with "Hour Of The Day" is quiet interesting.The median value are relatively higher at 7AM - 8AM and 5PM - 6PM. It can be attributed to regular school and office users at that time.
* Most of the outlier points are mainly contributed from "Working Day" than "Non Working Day". It is quiet visible from from figure 4.

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(18, 10)
fig.tight_layout(pad=3.0)
sn.boxplot(data=bike,y="count",orient="v",ax=axes[0][0])
sn.boxplot(data=bike,y="count",x="season",orient="v",ax=axes[0][1])
sn.boxplot(data=bike,y="count",x="hour",orient="v",ax=axes[1][0])
sn.boxplot(data=bike,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")

### Lets Remove Outliers In The Count Column

In [ ]:
bikeDataWithoutOutliers =  bike[np.abs(bike["count"]-bike["count"].mean())<=(3*bike["count"].std())] 

In [ ]:
print ("Shape Of The Before Ouliers: ",bike.shape)
print ("Shape Of The After Ouliers: ",bikeDataWithoutOutliers.shape)

## Correlation Analysis

One common to understand how a dependent variable is influenced by features (numerical) is to fibd a correlation matrix between them. Lets plot a correlation plot between "count" and ["temp","atemp","humidity","windspeed"].

* temp and humidity features has got positive and negative correlation with count respectively.Although the correlation between them are not very prominent still the count variable has got little dependency on "temp" and "humidity".
* windspeed is not gonna be really useful numerical feature and it is visible from it correlation value with "count"
* "atemp" is variable is not taken into since "atemp" and "temp" has got strong correlation with each other. During model building any one of the variable has to be dropped since they will exhibit multicollinearity in the data.
* "Casual" and "Registered" are also not taken into account since they are leakage variables in nature and need to dropped during model building.
* Regression plot in seaborn is one useful way to depict the relationship between two features. Here we consider "count" vs "temp", "humidity", "windspeed".

In [ ]:
corrMatt = bike[["temp","atemp","casual","registered","humidity","windspeed","count"]].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(20, 5)
sn.regplot(x="temp", y="count", data=bike,ax=ax1)
sn.regplot(x="windspeed", y="count", data=bike,ax=ax2)
sn.regplot(x="humidity", y="count", data=bike,ax=ax3)

### Visualizing Distribution Of Data

As it is visible from the below figures that "count" variable is skewed towards right. It is desirable to have Normal distribution as most of the machine learning techniques require dependent variable to be Normal. One possible solution is to take log transformation on "count" variable after removing outlier data points. After the transformation the data looks lot better but still not ideally following normal distribution.

In [ ]:
fig,axes = plt.subplots(ncols=2,nrows=2)
fig.set_size_inches(12, 10)
sn.distplot(bike["count"],ax=axes[0][0])
stats.probplot(bike["count"], dist='norm', fit=True, plot=axes[0][1])

# without outlier data frame
sn.distplot(np.log(bikeDataWithoutOutliers["count"]),ax=axes[1][0])
stats.probplot(np.log1p(bikeDataWithoutOutliers["count"]), dist='norm', fit=True, plot=axes[1][1])

### Visualizing Count Vs (Month,Season,Hour,Weekday,Usertype)

* It is quiet obvious that people tend to rent bike during summer season since it is really conducive to ride bike at that season.Therefore June, July and August has got relatively higher demand for bicycle.
* On weekdays more people tend to rent bicycle around 7AM-8AM and 5PM-6PM. As we mentioned earlier this can be attributed to regular school and office commuters.
* Above pattern is not observed on "Saturday" and "Sunday".More people tend to rent bicycle between 10AM and 4PM.
* The peak user count around 7AM-8AM and 5PM-6PM is purely contributed by registered user.


In [ ]:
fig,(ax1,ax2,ax3,ax4)= plt.subplots(nrows=4)
fig.set_size_inches(12,20)
bike
sortOrder = ["January","February","March","April","May","June","July","August","September","October","November","December"]
hueOrder = ["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]

monthAggregated = pd.DataFrame(bike.groupby("month")["count"].mean()).reset_index()
monthSorted = monthAggregated.sort_values(by="count",ascending=False)
sn.barplot(data=monthSorted,x="month",y="count",ax=ax1,order=sortOrder)
ax1.set(xlabel='Month', ylabel='Avearage Count',title="Average Count By Month")

hourAggregated = pd.DataFrame(bike.groupby(["hour","season"],sort=True)["count"].mean()).reset_index()
sn.pointplot(x="hour", y="count",hue="season", data=hourAggregated, join=True,ax=ax2)
ax2.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across Season",label='big')

hourAggregated = pd.DataFrame(bike.groupby(["hour","weekday"],sort=True)["count"].mean()).reset_index()
sn.pointplot(x="hour", y="count",hue="weekday",hue_order=hueOrder, data=hourAggregated, join=True,ax=ax3)
ax3.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across Weekdays",label='big')

hourTransformed = pd.melt(bike[["hour","casual","registered"]], id_vars=['hour'], value_vars=['casual', 'registered'])
hourAggregated = pd.DataFrame(hourTransformed.groupby(["hour","variable"],sort=True)["value"].mean()).reset_index()
sn.pointplot(x="hour", y="value",hue="variable",hue_order=["casual","registered"], data=hourAggregated, join=True,ax=ax4)
ax4.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across User Type",label='big')

## Filling 0's In windspeed Using Random Forest

In [ ]:
dataTrain = pd.read_csv("../input/bike-sharing-demand/train.csv")
dataTest = pd.read_csv("../input/bike-sharing-demand/test.csv")

### Combine Train And Test

In [ ]:
data = dataTrain.append(dataTest)
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

## Feature Engineering

In [ ]:
data["date"] = data.datetime.apply(lambda x : x.split()[0])

data["hour"] = data.datetime.apply(lambda x : x.split()[1].split(":")[0]).astype("int")

data["year"] = data.datetime.apply(lambda x : x.split()[0].split("-")[0])

data["weekday"] = data.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").weekday())

data["month"] = data.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").month)

### Random Forest Model To Predict 0's In Windspeed

In [ ]:
from sklearn.ensemble import RandomForestRegressor

dataWind0 = data[data["windspeed"]==0]
dataWindNot0 = data[data["windspeed"]!=0]

# initalize object
rfModel_wind = RandomForestRegressor()

# columns
windColumns = ["season","weather","humidity","month","temp","year","atemp"]

# fit the data
rfModel_wind.fit(dataWindNot0[windColumns], dataWindNot0["windspeed"])

# predict
wind0Values = rfModel_wind.predict(X= dataWind0[windColumns])

dataWind0["windspeed"] = wind0Values
data = dataWindNot0.append(dataWind0)
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

## Convert To Categorical Type

In [ ]:
categoricalFeatureNames = ["season","holiday","workingday","weather","weekday","month","year","hour"]

numericalFeatureNames = ["temp","humidity","windspeed","atemp"]

dropFeatures = ['casual',"datetime","date","registered"]

In [ ]:
for var in categoricalFeatureNames:
    data[var] = data[var].astype("category")

### Drop missing values


In [ ]:
data_df = data.copy(deep=True)

# drop unwanted columns
data_df  = data_df.drop(dropFeatures,axis=1)

In [ ]:
# drop missing values
data_df  = data_df.dropna()

data_df.shape

## Splitting predictor and target

In [ ]:
y = data_df['count']
x = data_df.drop('count', axis=1)

## Splitting Train And Test Data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size = .3, random_state=0)

## Using sklearn StackingRegressor

In [ ]:
# define the base models
level0 = list()
level0.append(('rig',Ridge()))
level0.append(('laso', Lasso()))
level0.append(('knn', KNeighborsRegressor()))
level0.append(('cart', DecisionTreeRegressor()))
level0.append(('svm', SVR(kernel='linear')))

# define meta learner model
level1 = LinearRegression()

# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

# fit the model on all available data
model.fit(x_train, y_train)

In [ ]:
# make prediction
y_predict = model.predict(x_test)

print('MAE : ', mean_absolute_error(y_test, y_predict))
print('MSE : ', mean_squared_error(y_test, y_predict))
print('R Squared : ', r2_score(y_test, y_predict))

## Using mlextend StackingCVRegressor

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

xgb = XGBRegressor()
lgbm = LGBMRegressor()
rf = RandomForestRegressor()
ridge = Ridge()
lasso = Lasso()
svr = SVR(kernel='linear')

stack = StackingCVRegressor(regressors=(ridge, lasso, svr, rf, lgbm, xgb),
                            meta_regressor=xgb, cv=12,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=False,
                            random_state=42)

stack.fit(x_train, y_train)

In [ ]:
# change the data type to int
x_test_2 = x_test.astype(int)

# feature column name (to avoid error)
x_test_2.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']

In [ ]:
# make prediction

y_predict_x = stack.predict(x_test_2)

In [ ]:
print('MAE : ', mean_absolute_error(y_test, y_predict_x))
print('MSE : ', mean_squared_error(y_test, y_predict_x))
print('R Squared : ', r2_score(y_test, y_predict_x))